In [1]:
import time

import numpy as np
# np.random.seed(1234)
from functools import reduce
import math as m

import scipy.io
from scipy.interpolate import griddata
from sklearn.preprocessing import scale
# from utils import augment_EEG, cart2sph, pol2cart

# import DNN frameworks
import tensorflow as tf
import keras

# import high level optimizers, models and layers
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import InputLayer

# for CNN
from keras.layers import Conv2D, MaxPooling2D
# for RNN
from keras.layers import LSTM

# for utility functionality
from keras.layers import Dense, Dropout, Flatten

from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
# imports tensorflow
# from keras import backend as K

ModuleNotFoundError: No module named 'sklearn'

In [2]:
from keras.datasets import imdb

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print len(mnist)
print type(mnist)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
3
<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>


# First Test Building the CNN

Here we want to create a function to build each layer of a CNN, say VGG style, or updated to be latest state-of-the art.

In [3]:
import numpy as np
import os

testfile = os.path.join('/Users/adam2392/Documents/pydata/output/mvar/la11_ictal/la11_ictal_mvarmodel.npz')
data = np.load(testfile)
print data.keys()

['adjmats', 'winsize', 'timepoints', 'includedchans', 'stepsize']


In [4]:
n_colors = 1
imsize=28

X_train= mnist.train.images
y_train = mnist.train.labels
# y_train=keras.utils.np_utils.to_categorical(y_train, num_classes=10)
X_test = mnist.test.images
y_test = mnist.test.labels

# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
# X_train /= 255
# X_test /= 255

# reshape into correct size to feed into NN
X_train = np.reshape(X_train, [-1, imsize, imsize, n_colors])
X_test = np.reshape(X_test, [-1, imsize, imsize, n_colors])

print X_train.shape
print y_train.shape

(55000, 28, 28, 1)
(55000, 10)


In [5]:
n_colors=1
imsize=X_train.shape[1]
print "im size is : ", imsize

def build_cnn(model, w_init=None, n_layers=(4,2,1),poolsize=(2,2),n_filters_first=32):    
    DEBUG=0
    
    # check for weight initialization -> apply Glorotuniform
    if w_init is None:
        w_init = [keras.initializers.glorot_uniform()] * sum(n_layers)
    
    # set up input layer of CNN
    model.add(InputLayer(input_shape=(imsize, imsize, n_colors)))
    # initialize counter
    count=0
    
    # add the rest of the hidden layers
    for idx, n_layer in enumerate(n_layers):
        for ilay in range(n_layer):
            model.add(Conv2D(n_filters_first*(2 ** idx), 
                             (3, 3),
                             input_shape=(imsize, imsize, n_colors),
                             kernel_initializer=w_init[count], activation='relu'))
            if DEBUG:
                print model.output_shape
                print idx, " and ", ilay
            count+=1

        # create a network at the end with a max pooling
        model.add(MaxPooling2D(pool_size=poolsize))
    return model

im size is :  28


In [84]:
w_init=None
model = Sequential()
model = build_cnn(model, w_init=w_init, n_layers=(2,1), n_filters_first=16)
print model.output

Tensor("max_pooling2d_33/MaxPool:0", shape=(?, 5, 5, 32), dtype=float32)


In [85]:
# to test training of this network, add a softmax at the end
model.add(Flatten())
print model.output_shape
model.add(Dense(800, activation='relu'))
print model.output_shape
model.add(Dense(10, activation='softmax'))
print model.output_shape

# Set Optimizer
# Nesterov Stochastic Gradient Descent
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# ADAM
ADAM = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=ADAM)

# cnn_config = model.get_config()
# display(cnn_config)

(None, 800)
(None, 800)
(None, 10)


In [86]:
n_colors = 1
imsize = 28

X_train= mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels

# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
# X_train /= 255
# X_test /= 255

print X_train.shape
print X_test.shape

# reshape into correct size to feed into NN
x_train = np.reshape(X_train, [-1, imsize, imsize, n_colors])
x_test = np.reshape(X_test, [-1, imsize, imsize, n_colors])

print model.output
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

print type(X_train)
print type(y_train)

# model.fit(X_train, y_train, batch_size=32, epochs=1)
model.fit(x_train[:100,:,:,:], y_train[:100,:], batch_size=20, epochs=1)

(55000, 784)
(10000, 784)
Tensor("dense_38/Softmax:0", shape=(?, 10), dtype=float32)
(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 2.2494


## Second Test Building The LSTM

We now want to build the RNN layers using LSTM. First show an example of the LSTM running sequence classification on the imdb dataset.

In [4]:
np.random.seed(7)
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17473536/17464789 [==============================] - 95s 5us/step


In [9]:
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(25000, 500)
(25000,)
(25000, 500)
(25000,)


In [8]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print X_train.shape
print X_test.shape

(25000, 500)
(25000, 500)


In [12]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 307s 12ms/step - loss: 0.5010 - acc: 0.7522 - val_loss: 0.3487 - val_acc: 0.8565
Epoch 2/2
25000/25000 [==============================] - 309s 12ms/step - loss: 0.4031 - acc: 0.8195 - val_loss: 0.3489 - val_acc: 0.8559


In [13]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.59%


## 3. Multi GPU Training Set Up

Using multiple gpus and/or just 1.

In [14]:
def build_convpool_lstm(model, nb_classes, grad_clip=110, imsize=32, n_colors=3, n_timewin=3):
    """
    Builds the complete network with LSTM layer to integrate time from sequences of EEG images.

    :param input_vars: list of EEG images (one image per time window)
    :param nb_classes: number of classes
    :param grad_clip:  the gradient messages are clipped to the given value during
                        the backward pass.
    :param imsize: size of the input image (assumes a square input)
    :param n_colors: number of color channels in the image
    :param n_timewin: number of time windows in the snippet
    :return: a pointer to the output of last layer
    """
    convnets = []
    w_init = None
    
    # Build 7 parallel CNNs with shared weights
    for i in range(n_timewin):
        if i == 0:
            convnet, w_init = build_cnn(input_vars[i], imsize=imsize, n_colors=n_colors)
        else:
            convnet, _ = build_cnn(input_vars[i], w_init=w_init, imsize=imsize, n_colors=n_colors)
        convnets.append(FlattenLayer(convnet))
    # at this point convnets shape is [numTimeWin][n_samples, features]
    # we want the shape to be [n_samples, features, numTimeWin]
    convpool = ConcatLayer(convnets)
    convpool = ReshapeLayer(convpool, ([0], n_timewin, get_output_shape(convnets[0])[1]))
    # Input to LSTM should have the shape as (batch size, SEQ_LENGTH, num_features)
    convpool = LSTMLayer(convpool, num_units=128, grad_clipping=grad_clip,
        nonlinearity=lasagne.nonlinearities.tanh)
    # We only need the final prediction, we isolate that quantity and feed it
    # to the next layer.
    convpool = SliceLayer(convpool, -1, 1)      # Selecting the last prediction
    # A fully-connected layer of 256 units with 50% dropout on its inputs:
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=256, nonlinearity=lasagne.nonlinearities.rectify)
    # And, finally, the output layer with 50% dropout on its inputs:
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=nb_classes, nonlinearity=lasagne.nonlinearities.softmax)
    return convpool

In [15]:
# number of gpus
G = 1
# we'll store a copy of the model on *every* GPU and then combine
# the results from the gradient updates on the CPU
with tf.device("/cpu:0"):
    # initialize the model
    model = cnn.build(width=32, height=32, depth=3, classes=10)

# make the model parallel
model = multi_gpu_model(model, gpus=G)

NameError: name 'cnn' is not defined

In [16]:
print tf.device("/cpu:0")